In [ ]:
import pandas as pd
from sklearn.metrics import r2_score,make_scorer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split,GridSearchCV
from xgboost import XGBRegressor
from metrics import partsMetrics,allMetrics

In [2]:
df=pd.read_csv('processed.csv')
print('Data shape:',df.shape)

Data shape: (467, 7)


In [3]:
X=df.iloc[:,:-1].values
y=df.iloc[:,-1].values
print(X.shape)
print(y.shape)

(467, 6)
(467,)


In [4]:
param_grid={
    'n_estimators': [None,10,50,200,300],   #default=100
    'max_depth': [None,2,6,10,20],
    'max_leaves': [None,10,20],
    'learning_rate': [None,.5,1],
    'subsample': [None,.5,.8],
    'reg_lambda': [None,.1,1,10,100],
    # 'reg_alpha': [None,.01,.1,1],
    'colsample_bytree': [None,.2,.5,.8],
    'gamma': [None,.1,.5,1],
    # 'min_child_weight': [None,.1,.5,1,2,4]    #default=1
}

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0)
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

def train_model(X_train,y_train):
    reg = XGBRegressor(random_state=0,n_jobs=2)
    grid=GridSearchCV(reg,param_grid,cv=5,scoring=make_scorer(r2_score),n_jobs=2)
    grid.fit(X_train, y_train)
    print(grid.best_params_)
    print(grid.best_score_)
    return grid

In [27]:
model=train_model(X_train,y_train)

{'colsample_bytree': 0.8, 'gamma': None, 'learning_rate': 1, 'max_depth': None, 'max_leaves': None, 'n_estimators': 300, 'reg_lambda': 100, 'subsample': None}
0.9486805565749405


In [5]:
best_params={'colsample_bytree': 0.8, 'gamma': None, 'learning_rate': 1, 'max_depth': None, 'max_leaves': None, 'n_estimators': 300, 'reg_lambda': 100, 'subsample': None}
reg=XGBRegressor(**best_params,random_state=0,n_jobs=2)
partsMetrics(df,reg)

train rmse: 0.27991232567386604
val rmse: 2.764372851687466
test rmse: 2.117698753182378

train si: 0.0256953481150257
val si: 0.23106815354477142
test si: 0.2063007117679637

train r2: 0.9992514626836105
val r2: 0.9424606564793905
test r2: 0.9642225043575966

train mape: 7.099584385786443
val mape: 13.576284776472622
test mape: 86.40003468867576


In [6]:
allMetrics(df,reg)

all rmse: 0.3412033349494472
all si: 0.030954571267694343
all r2: 0.9989609369569503
all mape: 10.114017700843801
